In [1]:
import torch
from torch_sparse import spmm

In [2]:
index = torch.tensor([[0, 0, 1, 2, 2],
                      [0, 2, 1, 0, 1]])
value = torch.Tensor([[1, 2, 4, 1, 3],
                      [3, 1, 5, 2, 4]])
matrix = torch.Tensor([[1, 4], [2, 5], [3, 6]])


In [3]:
out = spmm(index, value, 3, 3, matrix)


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/feng/.local/lib/python3.8/site-packages/torch_scatter/scatter.py", line 31, in scatter_add
                out: Optional[torch.Tensor] = None,
                dim_size: Optional[int] = None) -> torch.Tensor:
    return scatter_sum(src, index, dim, out, dim_size)
           ~~~~~~~~~~~ <--- HERE
  File "/home/feng/.local/lib/python3.8/site-packages/torch_scatter/scatter.py", line 12, in scatter_sum
                out: Optional[torch.Tensor] = None,
                dim_size: Optional[int] = None) -> torch.Tensor:
    index = broadcast(index, src, dim)
            ~~~~~~~~~ <--- HERE
    if out is None:
        size = src.size()
  File "/home/feng/.local/lib/python3.8/site-packages/torch_scatter/utils.py", line 13, in broadcast
    for _ in range(src.dim(), other.dim()):
        src = src.unsqueeze(-1)
    src = src.expand_as(other)
          ~~~~~~

pytorch geometric does not support batch sparse-dense

In [5]:
a = torch.sparse.FloatTensor(
    indices=torch.LongTensor([[0, 0, 0, 1, 1, 1],
                              [0, 1, 2, 0, 1, 2]]),
    values=torch.FloatTensor([[ 1.5901,  0.0183, -0.6146,  1.8061, -0.0112,  0.6302],
                              [ 1.5901,  0.0183, -0.6146,  1.8061, -0.0112,  0.6302]]),
    size=(2, 3, 2))

RuntimeError: indices and values must have same nnz, but got nnz from indices: 6, nnz from values: 2

This is also not working.

For the multiplication of the sparse matrix and batched dense matrices

In [9]:
indices = torch.LongTensor([[0, 0, 1, 2, 2],
                            [0, 2, 1, 0, 1]])
values = torch.tensor([1., 2., 4., 1., 3.])
s = torch.sparse.FloatTensor(indices=indices, values=values, size=(3, 3))
print(s)

tensor(indices=tensor([[0, 0, 1, 2, 2],
                       [0, 2, 1, 0, 1]]),
       values=tensor([1., 2., 4., 1., 3.]),
       size=(3, 3), nnz=5, layout=torch.sparse_coo)


In [10]:
m = torch.tensor([[1., 4., 2.], [2., 5., 2.], [3., 6., 4.]])
print(m)
result = torch.sparse.mm(s, m)
print(result)

tensor([[1., 4., 2.],
        [2., 5., 2.],
        [3., 6., 4.]])
tensor([[ 7., 16., 10.],
        [ 8., 20.,  8.],
        [ 7., 19.,  8.]])


In [ ]:
m = torch.tensor([[[1., 4., 2.], [2., 5., 2.], [3., 6., 4.]],
                  [[3., 1., 2.], [3., 2., 3.], [6., 2., 1.]]])
print(m)
result = torch.sparse.mm(s, m)
print(result)

batched sparse matrix not supported

tensor([[[1., 4., 2.],
         [2., 5., 2.],
         [3., 6., 4.]],

        [[3., 1., 2.],
         [3., 2., 3.],
         [6., 2., 1.]]])


RuntimeError: addmm: matrices expected, got 3D tensor

batched sparse matrix not supported